# 1. Database preparation

## 1.1. Import data

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="covid19",
    user="covid19",
    password="aluno")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("select * from agrega_bd")

# Retrieve query results
records = cur.fetchall()

In [2]:
records

[('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'Anti-HCV',
  'não reagente'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'Volume plaquetário médio',
  '11,8'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'Reticulócitos %',
  '1,20'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'C. Cetônicos, urina',
  'negativa'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'Urobilinogênio, urina',
  'inferior a 1,0 mg/dL'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'Células Epiteliais, urina',
  'raríssimas'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  82.0,
  'BR',
  'SP',
  'MMMM',
  'Sirio Libanes',
  'DHL',
  '531'),
 ('BDA95D6610B94CD8A7F2952E07B59342',
  'M',
  

## 1.2. Prepare dataframe

In [3]:
import pandas as pd

df = pd.DataFrame(records, columns=["id_paciente", "ic_sexo", "idade", "cd_pais", "cd_uf", "cd_municipio", "de_hospital", "de_analito", "de_resultado"])

df_piv = pd.pivot_table(df, index=["id_paciente", "ic_sexo", "idade", "cd_pais", "cd_uf", "cd_municipio", "de_hospital"], columns=["de_analito"], values=["de_resultado"] , aggfunc='first')

df_final = pd.DataFrame(df_piv.to_records())

df_final.columns = [hdr.replace("(", "") \
                     for hdr in df_final.columns]
df_final.columns = [hdr.replace(")", "") \
                     for hdr in df_final.columns]
df_final.columns = [hdr.replace("'de_resultado', '", "") \
                     for hdr in df_final.columns]
df_final.columns = [hdr.replace("'", "") \
                     for hdr in df_final.columns]

y = df_final.loc[:, 'test_covid']
x = df_final.pop("test_covid")


# df
# df[df['de_analito'] == 'test_covid']


# 2. Decision tree classifier

In [4]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y)

ValueError: could not convert string to float: 'DETECTADO'